In [42]:
import sys
sys.path.append("/Users/luoyifan/project/pineapple_scorecard/")
from utils import *

# 读取数据

In [3]:
data = pd.read_csv("data/data.csv")

## 转化label

In [4]:
# y用mob3 30+
y = "mob3"
data["y"] = data[y].apply(lambda x: get_label(x, 30, 0, -1))
data = data.loc[data["y"]!=-1]
data["loan_date2"] = data["loan_date"].str[:7]

## 剔除无关字段

In [5]:
data = data.drop(
    ['mob1', 'mob3', 'mob4', 'mob6',
       'dpd', 'dt'], axis=1
)

In [6]:
data.columns

Index(['id_no_des', 'user_id', 'fico_v2', 'pudao_jd', 'xysl_wxf', 'xysl_fcf',
       'xysl_qlf', 'tongdun_zx', 'tongdun_zr', 'tc_nyf', 'tc_zyhz', 'tc_k1502',
       'tc_zy', 'tc_a1502', 'zj_daiqian', 'fl_lxf', 'dxm_xdf_v1', 'dxm_xdf_v2',
       'hp_m3', 'hp_y', 'pudao_tx_5253', 'pudao_tx_5254', 'pudao_blz',
       'pudao_jzdz', 'br_scorecust13', 'br_scorecust14', 'fico_jzdz_h5',
       'fico', 'tx_tyf', 'mayi_dcf', 'rong360_zyf', 'rhjk_fr', 'rhjk_fr_v2',
       'hlz_score_3038', 'hlz', 'tongliantong_c', 'blz_tzf_10', 'blz_tzf_v1h1',
       'yl_qlf_a', 'ali_aly', 'loan_date', 'y', 'loan_date2'],
      dtype='object')

## 逾期率随时间的变化

In [68]:
overdue_rate_by_month = calculate_overdue_rate_by_month(data, "loan_date", "y").reset_index()

# 数据质量检测

## 缺失值

### 缺失率

In [8]:
missing_data, del_features_by_missing = calculate_missing_rate(
    data, "y"
).values()

### 缺失率随月份变化

In [9]:
missing_data_by_month = data.groupby(by=['loan_date2']).apply(lambda df: calculate_missing_rate(df, "y")["missing_data"]).reset_index().drop("level_1", axis=1)
missing_data_by_month = pd.pivot_table(missing_data_by_month, index="variable", columns="loan_date2", values=['missing_rate'], aggfunc="max").reset_index()
missing_data_by_month_columns = missing_data_by_month.columns.tolist()
missing_data_by_month.columns = [j[0] if i==0 else j[1] for i, j in enumerate(missing_data_by_month_columns)]
missing_data_by_month

,variable,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07
0,ali_aly,0.032695,0.035854,0.039464,0.029956,0.101781,0.144857,0.141361,0.136000,0.124939,0.126806,0.164326,0.181862
1,blz_tzf_10,0.774008,0.793564,0.666259,0.679398,0.672196,0.595315,0.582012,0.729184,0.755753,0.796938,0.818487,0.765768
2,blz_tzf_v1h1,0.733663,0.767302,0.677462,0.798139,0.783802,0.704199,0.707080,0.827451,0.897879,0.621580,0.231329,0.228307
3,br_scorecust13,1.000000,1.000000,1.000000,1.000000,0.447419,0.545748,0.660288,0.066137,0.031174,0.056428,0.129425,0.145642
4,br_scorecust14,1.000000,1.000000,1.000000,1.000000,0.447385,0.545760,0.681828,0.138830,0.103739,0.022746,0.011758,0.021263
5,dxm_xdf_v1,0.077407,0.078731,0.072221,0.135089,0.100981,0.103136,0.074269,0.002065,0.001186,0.001079,0.007495,0.013618
6,dxm_xdf_v2,0.413168,0.489883,0.625809,0.850746,0.810566,0.799822,0.785054,0.779126,0.705100,0.650089,0.716103,0.767919
7,fico,0.133018,0.142193,0.128820,0.117074,0.136500,0.179083,0.303385,0.307964,0.586972,0.847870,0.896339,0.878345
8,fico_jzdz_h5,0.874137,0.801402,0.771641,0.840714,0.809475,0.774843,0.802635,0.859170,0.920818,0.939802,0.929815,0.938026
9,fico_v2,1.000000,1.000000,1.000000,1.000000,0.867904,0.698771,0.715134,0.599639,0.401382,0.157353,0.150229,0.193330


In [10]:
data = data.drop(del_features_by_missing, axis=1)

## 高频次项占比

In [11]:
mode_data, del_features_by_mode = calculate_mode_rate(data, "y").values()

In [12]:
del_features_by_mode

[]

In [13]:
data = data.drop(del_features_by_mode, axis=1)

## 获取需要的特征

In [14]:
need_features = data.drop(['id_no_des', 'user_id', 'y', 'loan_date2', 'loan_date'], axis=1).columns.tolist()

In [15]:
need_features

['fico_v2',
 'xysl_wxf',
 'xysl_fcf',
 'xysl_qlf',
 'tongdun_zx',
 'tongdun_zr',
 'tc_nyf',
 'tc_zyhz',
 'tc_k1502',
 'tc_zy',
 'tc_a1502',
 'zj_daiqian',
 'fl_lxf',
 'dxm_xdf_v1',
 'dxm_xdf_v2',
 'hp_m3',
 'hp_y',
 'pudao_tx_5253',
 'pudao_tx_5254',
 'pudao_blz',
 'br_scorecust13',
 'br_scorecust14',
 'fico_jzdz_h5',
 'fico',
 'tx_tyf',
 'mayi_dcf',
 'rong360_zyf',
 'rhjk_fr',
 'rhjk_fr_v2',
 'hlz_score_3038',
 'hlz',
 'tongliantong_c',
 'blz_tzf_10',
 'blz_tzf_v1h1',
 'ali_aly']

# 划分数据集

In [16]:
dev = data.loc[data["loan_date2"].isin(["2022-12", "2023-01"]), need_features+['y']]
oot = data.loc[data["loan_date2"] == "2023-02", need_features+['y']]

# woe分箱

In [17]:
res = get_iv_psi_data(dev, oot, "y", badratediff=0)

 26%|███████████████████████████████                                                                                          | 9/35 [00:17<00:41,  1.60s/it]

tc_k1502 error!


 31%|█████████████████████████████████████▋                                                                                  | 11/35 [00:24<00:56,  2.35s/it]

tc_a1502 error!


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [01:18<00:00,  2.26s/it]


In [18]:
dev = dev.drop(['tc_k1502', 'tc_a1502'], axis=1)
oot = oot.drop(['tc_k1502', 'tc_a1502'], axis=1)

In [19]:
res['del_features_by_ivpsi']

['br_scorecust14',
 'hp_y',
 'tongliantong_c',
 'pudao_tx_5253',
 'hlz',
 'mayi_dcf',
 'pudao_tx_5254',
 'hlz_score_3038',
 'fico_v2',
 'fl_lxf',
 'dxm_xdf_v2',
 'pudao_blz',
 'tc_nyf',
 'zj_daiqian',
 'tc_zyhz',
 'rhjk_fr']

In [24]:
dev = dev.drop(res['del_features_by_ivpsi'], axis=1)
oot = oot.drop(res['del_features_by_ivpsi'], axis=1)

In [20]:
dev_woe = sc.woebin_ply(dev, res['bins'], print_info=False)
oot_woe = sc.woebin_ply(oot, res['bins'], print_info=False)

# 相关性筛选

In [35]:
dev.shape

(306760, 34)

In [22]:
iv_psi_data = res['iv_psi_data']

iv_psi_data2 = iv_psi_data.loc[~iv_psi_data['variable'].isin(res['del_features_by_ivpsi'])]
del_features_by_corr = select_feature_by_corr_woe(iv_psi_data2, 'train_total_iv', dev_woe)

In [24]:
len(del_features_by_corr)

1

In [27]:
iv_psi_data2.loc[iv_psi_data2['variable']=='tongdun_zx']

,variable,train_total_iv,oot_total_iv,train_oot_psi
3,tongdun_zx,0.102737,0.097244,0.099453


In [28]:
dev_woe = dev_woe.drop(del_features_by_corr, axis=1)
oot_woe = oot_woe.drop(del_features_by_corr, axis=1)

# 逐步回归

In [32]:
from stepwise import StepWise
step_wise = StepWise(label="y")
X = dev_woe.drop("y", axis=1)
y = dev_woe["y"]
need_features = step_wise.fit(X, y)


-------------------------------------向前挑选变量------------------------------------
向前添加了 'blz_tzf_v1h1_woe' 
向前的aic的分数107286.519878
向后没有足够的特征集合
-------------------------------------向前挑选变量------------------------------------
向前添加了 'hp_m3_woe' 
向前的aic的分数106612.508662
-------------------------------------向后挑选变量------------------------------------
后向后没有挑选到合适的变量
-------------------------------------向前挑选变量------------------------------------
向前添加了 'blz_tzf_10_woe' 
向前的aic的分数106281.696848
-------------------------------------向后挑选变量------------------------------------
后向后没有挑选到合适的变量
-------------------------------------向前挑选变量------------------------------------
向前添加了 'xysl_qlf_woe' 
向前的aic的分数106010.492574
-------------------------------------向后挑选变量------------------------------------
后向后没有挑选到合适的变量
-------------------------------------向前挑选变量------------------------------------
向前添加了 'br_scorecust14_woe' 
向前的aic的分数105816.068554
-------------------------------------向后挑选变量-----------------------------

# 建立模型

In [40]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split



X = dev_woe[need_features]
X = sm.add_constant(X)
y = dev_woe['y']

X_oot = oot_woe[need_features]
X_oot = sm.add_constant(X_oot)
y_oot = oot_woe['y']


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, stratify=y, test_size=0.3)
model = sm.Logit(y_train, X_train).fit(method="lbfgs", disp=-1)

print(model.summary2())

                           Results: Logit
Model:               Logit             Method:            MLE       
Dependent Variable:  y                 Pseudo R-squared:  0.041     
Date:                2023-07-10 15:43  AIC:               73551.9280
No. Observations:    214732            BIC:               73778.0252
Df Model:            21                Log-Likelihood:    -36754.   
Df Residuals:        214710            LL-Null:           -38341.   
Converged:           1.0000            LLR p-value:       0.0000    
No. Iterations:      26.0000           Scale:             1.0000    
--------------------------------------------------------------------
                    Coef.  Std.Err.     z     P>|z|   [0.025  0.975]
--------------------------------------------------------------------
const              -3.0943   0.0110 -280.1763 0.0000 -3.1160 -3.0727
blz_tzf_v1h1_woe    0.5080   0.0386   13.1473 0.0000  0.4323  0.5837
hp_m3_woe           0.4517   0.0412   10.9691 0.0000  0.3710 

In [52]:
# cut-off决策表
train_proba = model.predict(X_train)
test_proba = model.predict(X_test)
oot_proba = model.predict(X_oot)

proba2score = Proba2Score(point=600, odds=0.04, pdo=100)
res = pd.DataFrame()
ks_collection = []
for tmp in [
    ("train", y_train, train_proba),
    ("test", y_test, test_proba),
    ("oot", y_oot, oot_proba)
]:
    auc, ks = get_auc_ks(tmp[1], tmp[2], is_print=True).values()
    ks_collection.append({
        "sample_type": tmp[0],
        "cnt": tmp[1].shape[0],
        "bad": tmp[1].sum(),
        "bad_rate": tmp[1].mean(),
        "auc": auc,
        "ks": ks
    })
    res = pd.concat(
        [
            res, pd.DataFrame({
                "sample_type": tmp[0],
                "y": tmp[1],
                "score": proba2score.proba2score(tmp[2])
            })
        ]
    )


auc: 
 0.6616368095574254
ks: 
 0.22655325682618593
auc: 
 0.6587956779763007
ks: 
 0.2229451719415093
auc: 
 0.6477670777934085
ks: 
 0.20999972085423468


In [53]:
my_type = pd.CategoricalDtype(
  categories=["train","test", "oot"],
  ordered=True
)
res["sample_type"] = res["sample_type"].astype(my_type)

In [63]:
pd.DataFrame(ks_collection)

,sample_type,cnt,bad,bad_rate,auc,ks
0,train,214732,9316,0.043384,0.661637,0.226553
1,test,92028,3993,0.043389,0.658796,0.222945
2,oot,146009,6828,0.046764,0.647767,0.210000


# cut-off-data

## 等频

In [54]:
pct = list(range(10, 100, 10))
sp = sorted(
            list(set(np.percentile(res["score"].tolist(), pct)))
        )
cut_off_data1 = res.groupby(by=['sample_type']).apply(lambda x: calculate_cut_off_data(x, "y", "score", sp, is_plot=False)).reset_index()
cut_off_data1['type'] = "等频"

## 等宽

In [55]:
res["score"].describe()

count    452769.000000
mean        599.903755
std          81.242061
min         249.265322
25%         549.504732
50%         609.902351
75%         658.615096
max         822.604806
Name: score, dtype: float64

In [60]:
# pct = list(range(10, 100, 10))
# sp = sorted(
#             list(set(np.percentile(res["score"].tolist(), pct)))
#         )
sp = list(range(400, 750, 30))
cut_off_data2 = res.groupby(by=['sample_type']).apply(lambda x: calculate_cut_off_data(x, "y", "score", sp, is_plot=False)).reset_index()
cut_off_data2['type'] = "等宽"

# 模型报告

In [69]:
overdue_rate_by_month

,loan_date,计数,坏,坏样本率
0,2022-08,92950,5639,0.060667
1,2022-09,88415,5319,0.060159
2,2022-10,89171,5508,0.061769
3,2022-11,148820,6563,0.044100
4,2022-12,147582,6271,0.042492
5,2023-01,159178,7038,0.044215
6,2023-02,146009,6828,0.046764
7,2023-03,138500,4072,0.029401
8,2023-04,164432,2180,0.013258
9,2023-05,157563,241,0.001530


In [ ]:
class ModelReport:

    def __init__(self):
        pass

    def fit(self):
        # 逾期率随时间的分布
        # 逾期率随样本集合的分布
        # 缺失值的分布
        # 缺失值随时间的分布
        # 模型效果
        # 入模特征效果IV、PSI
        # 入模特征的相关性
        # 入模特征的分箱逾期率
        # cut-off
        pass